<a href="https://colab.research.google.com/github/chinghenglin/TibaMe_project2/blob/main/yolo_v8_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!scp -r /content/gdrive/MyDrive/OverDRIVE_site54991_TibaMe_AI233_G4/YOLO訓練資料/影像分類 /content/

In [3]:
import os

# 資料夾名稱
folders = [f.name for f in os.scandir('/content/影像分類') if os.path.isdir(f)]
print(folders)
print(len(folders))

['C03W', 'A11W', 'A05S', 'A08N', 'D01N', 'A03E', 'C01E', 'B01S', 'A13W', 'E08N', 'A08S', 'A21W', 'C09W', 'C12W', 'A14W', 'D03S', 'E08S', 'E01N', 'C15S', 'A21E', 'A07E', 'C12E', 'E02N', 'A19S', 'E07N', 'A22N', 'C15N', 'A20N', 'A05N', 'A04N', 'A15E', 'A06W', 'C11W', 'A01N', 'A02E', 'C06E', 'A12E', 'E03N', 'C18E', 'C11E', 'C10W', 'A12W', 'C07N', 'A09E', 'C08E', 'C04N', 'A09W', 'A11E', 'E05S', 'E06N', 'C14W', 'A10E', 'A20S', 'A07W', 'A10W', 'A13E', 'A03W', 'C02W', 'C04S', 'C17S', 'C10E', 'E06S', 'A02W', 'C08W', 'C13W', 'A18W', 'D03N', 'A17W', 'C06W', 'C05S', 'A06E', 'E04N', 'C07S', 'B02S', 'B02N', 'A14E', 'A04S', 'A16N', 'A15W', 'E03S', 'C01W', 'E07S', 'D02E', 'C14E', 'C19S', 'E04S', 'C09E', 'C16W', 'B01N', 'E02S', 'C13E', 'E01S', 'C16E', 'D01S', 'E05N']
95


In [4]:
# 建立 test/ train/ 和 val/ 資料夾
os.makedirs("data/tpstn/test", exist_ok=True)
os.makedirs("data/tpstn/train", exist_ok=True)
os.makedirs("data/tpstn/val", exist_ok=True)

In [5]:
import shutil
import glob

for folder in folders:
    # 建立 test/ 資料夾下的子資料夾
    os.makedirs(os.path.join("data/tpstn/test", folder), exist_ok=True)

    test_files  = [os.path.basename(fn) for fn in glob.glob(f'影像分類/{folder}/*_HH_*')]

    for file in test_files:
        src_path = os.path.join("影像分類", folder, file)
        dest_path = os.path.join("data/tpstn/test", folder, file)
        shutil.move(src_path, dest_path)


In [6]:
import random

# 用來設定訓練資料和驗證資料比例的變數
train_ratio = 0.8
val_ratio = 0.2

# 依序處理每個資料夾
for folder in folders:
    # 建立 train/ 和 val/ 資料夾下的子資料夾
    os.makedirs(os.path.join("data/tpstn/train", folder), exist_ok=True)
    os.makedirs(os.path.join("data/tpstn/val", folder), exist_ok=True)

    # 取得該資料夾下所有影像檔案的檔名列表
    files = os.listdir(f'影像分類/{folder}')

    # 計算要從該資料夾中挑選多少筆資料放進驗證資料集
    num_val_samples = int(len(files) * val_ratio)

    # 隨機挑選影像檔案放進驗證資料集
    val_files = random.sample(files, num_val_samples)

    # 將影像檔案從原本的資料夾移動到驗證資料集的子資料夾
    for file in val_files:
        src_path = os.path.join('影像分類', folder, file)
        dest_path = os.path.join("data/tpstn/val", folder, file)
        shutil.move(src_path, dest_path)

    # 剩餘的影像檔案就是訓練資料集
    train_files = [file for file in files if file not in val_files]

    # 將影像檔案從原本的資料夾移動到訓練資料集的子資料夾
    for file in train_files:
        src_path = os.path.join('影像分類', folder, file)
        dest_path = os.path.join("data/tpstn/train", folder, file)
        shutil.move(src_path, dest_path)

print("影像資料已成功分割並移動到 train/ 和 val/ 資料夾中。")

影像資料已成功分割並移動到 train/ 和 val/ 資料夾中。


In [7]:
!nvidia-smi

Sun Jul 30 08:21:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# Pip install method (recommended)
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.145 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/78.2 GB disk)


In [9]:
data_dir = '/content/data/tpstn'

In [10]:
from ultralytics import YOLO
from IPython.display import display, Image

In [17]:
# Load a model
model = YOLO('yolov8n-cls.pt')

# Use the model
results = model.train(data = data_dir, epochs = 10, imgsz = 512)

Ultralytics YOLOv8.0.145 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/data/tpstn, epochs=10, patience=50, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False

In [18]:
!scp -r /content/runs /content/gdrive/MyDrive/YOLO_v8/20230730/